In [ ]:
!sudo pip install python-dotenv

In [3]:
%%bash

bucketname="$(echo $USER"-sql-workshop"|cut -d "-" -f2-)"
echo $bucketname

cp /etc/skel/.env ~/
echo "BUCKETNAME="$bucketname >> ~/.env

aws s3api create-bucket --bucket $bucketname --region us-east-1
aws s3api list-buckets | grep $bucketname 

arhodes-sql-workshop
/arhodes-sql-workshop
BUCKETS	2021-08-30T19:29:32.000Z	arhodes-sql-workshop


In [4]:
from dotenv import load_dotenv
load_dotenv()

True

In [6]:
import os 
os.getenv('BUCKETNAME')

'arhodes-sql-workshop'

In [7]:
import os
os.environ['ATHENA_USER'] = os.getenv('AWS_ACCESS_KEY_ID')
os.environ['ATHENA_PASSWORD'] = os.getenv('AWS_SECRET_ACCESS_KEY')
os.environ['ATHENA_REGION']= 'us-east-1'
os.environ['ATHENA_STAGING']= os.getenv('BUCKETNAME')
os.environ['ATHENA_DATABASE']='sra'

In [8]:
import pyathena
import pandas as pd


#This line of code gives the connection information to AWS, this variable is used in the sql commands

athena_conn = pyathena.connect(aws_access_key_id=os.environ['ATHENA_USER'], ##credentials of aws_access_key_id
                 aws_secret_access_key=os.environ['ATHENA_PASSWORD'], ##credentials of aws_secret_access_key
                 s3_staging_dir=os.environ['ATHENA_STAGING'], ##where the athena query result saved - checked in S3 ,
                 region_name=os.environ['ATHENA_REGION']) ##the region you set for Athena




In [9]:
%load_ext sql

In [53]:
import sqlalchemy
from urllib.parse import quote_plus
from urllib.parse import quote
from sqlalchemy.engine import create_engine


AWS_ACCESS_KEY = os.environ['ATHENA_USER']
AWS_SECRET_KEY = os.environ['ATHENA_PASSWORD']
SCHEMA_NAME = os.environ['ATHENA_DATABASE']
S3_STAGING_DIR = "s3://" + os.environ['ATHENA_STAGING']
AWS_REGION = "us-east-1"


conn_str = (
    "awsathena+rest://{aws_access_key_id}:{aws_secret_access_key}@"
    "athena.{region_name}.amazonaws.com/?"
    "{schema_name}s3_staging_dir{s3_staging_dir}&work_group=primary"
)



# Create the SQLAlchemy connection. Note that you need to have pyathena installed for this.
connection = conn_str.format(
        aws_access_key_id=quote_plus(AWS_ACCESS_KEY),
        aws_secret_access_key=quote_plus(AWS_SECRET_KEY),
        region_name=AWS_REGION,
        schema_name=SCHEMA_NAME,
        s3_staging_dir=quote_plus(S3_STAGING_DIR),
    )

engine = create_engine(connection)

In [54]:
%config SqlMagic.displaycon = False

In [56]:
%%sql

SELECT acc, librarysource, bioproject, geo_loc_name_country_calc
FROM sra.metadata
WHERE organism = 'Homo sapiens'
LIMIT 5

Done.


acc,librarysource,bioproject,geo_loc_name_country_calc
SRR1006444,GENOMIC,PRJNA222325,None
SRR1006161,GENOMIC,PRJNA222325,None
SRR7685716,GENOMIC,PRJNA485420,None
SRR7685710,GENOMIC,PRJNA485420,None
SRR7685648,GENOMIC,PRJNA485420,None


In [ ]:
#Use this code block by removing the "#" in front of the following commands 
#After the workshop to set up credentials for your AWS Account To use this Notebook
#This has already been done on the workshop server

#%env AWS_ACCESS_KEY_ID=<your_AWS_Access ID>
#%env AWS_SECRET_ACCESS_KEY=<your AWS Secret>
#%env BUCKETNAME=<your staging bucket name>


#!aws configure set aws_access_key_id "$AWS_ACCESS_KEY_ID"
#!aws configure set aws_secret_access_key "$AWS_SECRET_ACCESS_KEY" 
#!aws configure set region "us-east-1" 
#!aws configure set output "text"


#!sudo pip install pyathena 
#!sudo pip install ipython-sql
#!pip install sqlalchemy

